In [1]:
from lonboard import Map, SolidPolygonLayer
from lonboard.basemap import CartoBasemap
from lonboard.colormap import apply_continuous_cmap
import xradar as xd
import xarray as xr
import numpy as np
from pyxlma import coords
import cmweather
import geopandas as gpd

from arro3.core import Array, Field, fixed_size_list_array, list_array, Table, Schema
from geoarrow.rust import core as geoarrow

import geoarrow.pyarrow as ga

In [2]:
import requests
from os import path

if not path.exists("KLBB20240922_034901_V06"):
    radar_data = requests.get(
        "https://noaa-nexrad-level2.s3.amazonaws.com/2024/09/22/KLBB/KLBB20240922_034901_V06"
    )
    with open("KLBB20240922_034901_V06", "wb") as f:
        f.write(radar_data.content)

In [3]:
rdr = xr.open_dataset(
    "KLBB20240922_034901_V06", group="sweep_0", engine="nexradlevel2", chunks="auto"
)

In [4]:
reflec_i_want = rdr.DBZH.data.flatten()
reflec_i_want = reflec_i_want[reflec_i_want > -10]
reflectivity_normalized = reflec_i_want / 90 + 1 / 9

In [5]:
rcs = coords.RadarCoordinateSystem(
    rdr.latitude.data.item(), rdr.longitude.data.item(), rdr.altitude.data.item()
)

In [6]:
range2d, el2d = np.meshgrid(
    coords.centers_to_edges(rdr.range.data), coords.centers_to_edges(rdr.elevation.data)
)
range2d, az2d = np.meshgrid(
    coords.centers_to_edges(rdr.range.data), coords.centers_to_edges(rdr.azimuth.data)
)

In [7]:
lon, lat, alt = rcs.toLonLatAlt(range2d, az2d, el2d)

In [8]:
lon_edge = np.array(lon).reshape(range2d.shape)
lat_edge = np.array(lat).reshape(range2d.shape)
alt_edge = np.array(alt).reshape(range2d.shape)

In [9]:
# bottom-left: (i, j)
bl_lon = lon_edge[:-1, :-1]
bl_lat = lat_edge[:-1, :-1]
# bl_alt = alt_edge[:-1, :-1]

# bottom-right: (i+1, j)
br_lon = lon_edge[1:, :-1]
br_lat = lat_edge[1:, :-1]
# br_alt = alt_edge[1:, :-1]

# top-right: (i+1, j+1)
tr_lon = lon_edge[1:, 1:]
tr_lat = lat_edge[1:, 1:]
# tr_alt = alt_edge[1:, 1:]

# top-left: (i, j+1)
tl_lon = lon_edge[:-1, 1:]
tl_lat = lat_edge[:-1, 1:]
# tl_alt = alt_edge[:-1, 1:]

In [10]:
lons_to_poly = np.array(
    [bl_lon.flatten(), br_lon.flatten(), tr_lon.flatten(), tl_lon.flatten()]
)
lats_to_poly = np.array(
    [bl_lat.flatten(), br_lat.flatten(), tr_lat.flatten(), tl_lat.flatten()]
)
# alts_to_poly = np.array([bl_alt.flatten(), br_alt.flatten(), tr_alt.flatten(), tl_alt.flatten()])

In [11]:
poly_coords = np.stack([lons_to_poly, lats_to_poly], axis=-1).transpose(1, 0, 2)

In [12]:
poly_coords.shape

(1319040, 4, 2)

In [13]:
bad_points_mask_bool = (rdr.DBZH.data.flatten() > -10).compute().flatten()
bad_points_mask = np.argwhere(bad_points_mask_bool).flatten()
poly_coords = poly_coords[bad_points_mask, :, :]

In [14]:
poly_coords[:, 0, :]


array([[-101.81418595,   33.6721714 ],
       [-101.81418879,   33.67442525],
       [-101.81419164,   33.67667911],
       ...,
       [-101.84229563,   36.22572718],
       [-101.84232103,   36.2279762 ],
       [-101.84234643,   36.23022521]])

In [15]:
poly_coords_closed = np.concat((poly_coords, poly_coords[:, 0:1, :]), axis=1)
print(poly_coords_closed.shape)
print(np.ravel(poly_coords_closed).shape)

(366143, 5, 2)
(3661430,)


In [16]:
poly_coords_arrow = fixed_size_list_array(
    Array.from_numpy(np.ravel(poly_coords_closed)), 2
)

In [17]:
ring_offsets = Array.from_numpy(
    np.arange(0, (poly_coords_closed.shape[0] + 1) * 5, 5, dtype=np.int32)
)

In [18]:
arrow_rings = list_array(ring_offsets, poly_coords_arrow)

In [19]:
len(ring_offsets)

366144

In [20]:
geom_offsets = Array.from_numpy(
    np.arange(poly_coords_closed.shape[0] + 1, dtype=np.int32)
)

In [21]:
arrow_geoms = list_array(geom_offsets, arrow_rings)

In [22]:
extension_metadata = {"ARROW:extension:name": "geoarrow.polygon"}
field = Field("geometry", arrow_geoms.type, nullable=True, metadata=extension_metadata)

In [23]:
schema = Schema([field])

In [24]:
table = Table.from_arrays([arrow_geoms], schema=schema)

In [25]:
layer = SolidPolygonLayer(
    table=table,
    get_fill_color=apply_continuous_cmap(
        reflectivity_normalized, cmap=cmweather.cm_colorblind.ChaseSpectral, alpha=0.5
    ),
)

/Users/kyle/github/kylebarron/lonboard-radar-example/.pixi/envs/default/lib/python3.12/site-packages/lonboard/_geoarrow/ops/reproject.py:23: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


In [26]:
m = Map(layer, basemap_style=CartoBasemap.Positron)

In [27]:
m

Map(basemap_style=<CartoBasemap.Positron: 'https://basemaps.cartocdn.com/gl/positron-gl-style/style.json'>, la…